In [ ]:
!pip install feature_engine

In [ ]:
pip install statsmodels

**Python 包**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
# Plots
import seaborn as sns
import matplotlib.pyplot as plt

from feature_engine.imputation import MeanMedianImputer

import statsmodels.api as sm
import pylab

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**数据基本情况检查**

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train.head()

In [ ]:
pd.DataFrame(data = [train.isna().sum()/train.shape[0]*100, test.isna().sum()/test.shape[0]*100], 
             index=["Train Null (%)", "Test Null (%)"]).T.style.background_gradient(cmap='summer_r')

In [ ]:
train_data = train.drop(['Alley','PoolQC','Fence','MiscFeature','FireplaceQu'],axis = 1)
test_data = test.drop(['Alley','PoolQC','Fence','MiscFeature','FireplaceQu'],axis = 1)

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(train['SalePrice'], color="b")
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="SalePrice")
ax.set(title="SalePrice Distribution")
sns.despine(trim=True, left=True)
plt.show()

In [ ]:
# 斜度 和 曲度
print("Skewness: %f" % train['SalePrice'].skew())
print("Kurtosis: %f" % train['SalePrice'].kurt())

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(np.log(train['SalePrice']), color="b")
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="log SalePrice")
ax.set(title="log SalePrice Distribution")
sns.despine(trim=True, left=True)
plt.show()

In [ ]:
# 斜度 和 曲度
print("Skewness: %f" % np.log(train['SalePrice']).skew())
print("Kurtosis: %f" % np.log(train['SalePrice']).kurt())

**所以我们考虑对lnY回归**

In [ ]:
# Finding numeric features
# 这样分出来的不全是数值型的
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num = []
cat = []
for i in train_data.columns:
    if (train_data[i].dtype in numeric_dtypes)&(i != 'SalePrice'):
            num.append(i)  
    elif i != 'SalePrice': #elif train_data[i].dtype=='object'
            cat.append(i) 

In [ ]:
#对num、cat进行人工校对,可能还有那天晚上后来没注意了
change = ['MSSubClass','OverallQual']
for c in change:
    num.remove(c)
    cat.insert(-1,c)

**各变量分布情况**

In [ ]:
def num_dist(data, var):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    sns.histplot(data=data, x=var, kde=True, ax=ax[0])
    sns.boxplot(data=data, x=var, ax=ax[1])
    ax[0].set_title(f"{var} Distribution Histogram")
    ax[1].set_title(f"{var} Distribution Boxplot")

    plt.show()
for var in num:
    num_dist(train, var)

In [ ]:
def cat_dist(data, var):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    train_data[var].value_counts().plot(kind="pie", explode=[0.05 for x in data[var].dropna().unique()], autopct='%1.1f%%', ax=ax[0], shadow=True)
    ax[0].set_title(f"{var} Pie Chart")
    ax[0].set_ylabel('')

    count = sns.countplot(x=var, data=train_data, ax=ax[1])
    for bar in count.patches:
        count.annotate(format(bar.get_height()),
            (bar.get_x() + bar.get_width() / 2,
            bar.get_height()), ha='center', va='center',
            size=11, xytext=(0, 8),
            textcoords='offset points')
    ax[1].set_title(f"{var} Bar Chart")
    plt.show()

for c in cat:
    cat_dist(train,c)


**缺失值补齐**

https://www.kaggle.com/code/dansbecker/handling-missing-values

连续型：

1. 缺失值占比很少很少的用平均值或者中位数？？？0%~10% sklearn.impute.SimpleImputer

2. 缺失值稍微大的用????  10%以上的50%以下的

3. 50%以上的扔了吧

分类:

并不是很多，把缺的行删了

In [ ]:
# 连续型，只有一个用中位数补齐
median_imputer = MeanMedianImputer(imputation_method="median")
train_data[num] = median_imputer.fit_transform(train_data[num])
test_data[num] = median_imputer.transform(test_data[num])

In [ ]:
# 合并数据进行数据处理
train_data.insert(train_data.shape[1],'lable',np.ones(train_data.shape[0]))
test_data.insert(test_data.shape[1],'lable',np.zeros(test_data.shape[0]))
data = pd.concat([train_data,test_data],axis = 0)

In [ ]:
# 分类
missing = (data.isna().sum()!=0)[data.isna().sum()!=0].index.values
cat_missing = []
for m in missing:
    if m in cat:
        cat_missing.append(m)
data.dropna(subset = cat_missing,axis=0,inplace = True)

In [ ]:
# 删去变量，需要有个人写一下解释原因，结合图像和数据分析
drop_ = ['Id','Neighborhood','Condition1','Condition2','Exterior1st','Exterior2nd','BsmtQual','BsmtFinType1','BsmtFinSF1',
        'BsmtFinType2','BsmtFinSF2','Heating','GarageYrBlt','GarageFinish','GarageCars','MoSold','YrSold']
data.drop(drop_,axis = 1,inplace = True)

In [ ]:
# 更新变量分类
for d in drop_:
    for c in cat:
        if d==c:
            cat.remove(c)
    for n in num:
        if d==n:
            num.remove(n)
data = data.reset_index()

**特征工程** 

1. YearBuilt 分箱

In [ ]:
def box_yearbuild(col):
    peroid1 = [0]*len(col)
    peroid2 = [0]*len(col)
    for i in range(0,len(col)):
        if col[i]<=1950:
            peroid1[i] = 1
        elif col[i] > 1980:
            peroid2[i] = 1
    per = pd.DataFrame(peroid1,columns = ['YearBuilt_Before1950'])
    per.insert(1,'YearBuilt_After1980',peroid2)
    return per
data = pd.concat([data,box_yearbuild(data['YearBuilt'])],axis = 1)
data.drop('YearBuilt', axis = 1,inplace = True)

2. YearRemodAdd 分箱

In [ ]:
def box_yearremod(col):
    peroid = [0]*len(col)
    for i in range(0,len(col)):
        if col[i]<=1990:
            peroid[i] = 1
    per = pd.DataFrame(peroid,columns = ['YearRemodel_before1990'])
    return per
data = pd.concat([data,box_yearremod(data['YearRemodAdd'])],axis = 1)
data.drop('YearRemodAdd', axis = 1,inplace = True)

3.  比例构造

    BsmtUnfSF/TotalBsmtSF 未完成的地下室占比

    LowQualFinSF/GrLivArea 

In [ ]:
def ratio(data,num,den,new_name):
    ratio = data[num]/data[den]
    data.insert(data.shape[1],new_name,ratio)
    data.drop([num, den],axis = 1, inplace = True)
ratio(data,'BsmtUnfSF','TotalBsmtSF','UnfinishedBsm_ratio')
ratio(data,'LowQualFinSF','GrLivArea','LowQuality_ratio')

4. 卫生间面积合并

In [ ]:
def Bath_combine(data):
    Bath = 0.5*(0.6*data['HalfBath']+0.4*data['BsmtHalfBath'])+0.6*data['FullBath']+0.4*data['BsmtFullBath']
    data.insert(data.shape[1],'Bath_total',Bath)
    data.drop(['HalfBath', 'BsmtHalfBath', 'FullBath', 'BsmtFullBath'],axis = 1, inplace = True)
Bath_combine(data)

5. 门廊合并

In [ ]:
def Porch_combine(data):
    porch = [0]*data.shape[0]
    for a in data.columns:
        if 'Porch' in a:
            porch += data[a]
            data.drop(a, axis = 1, inplace = True)
    data.insert(data.shape[1],'Porch',porch)
Porch_combine(data)

6. GarageQual & Cond 比较留

In [ ]:
def compare(data):
    dic = {'Ex':6, 'Gd':5, 'TA':4, 'Fa':3, 'Po':2, 'NA':1}
    Garage = []
    for i in range(data.shape[0]):
        if data['GarageQual'].map(dic)[i] <= data['GarageCond'].map(dic)[i]:
            Garage.append(data['GarageQual'][i])
        else: 
            Garage.append(data['GarageCond'][i])
    data.insert(data.shape[1],'Garagelevel',Garage)
    data.drop(['GarageQual','GarageCond'], axis = 1, inplace = True)
compare(data)

**重新提取数值型和分类变量**

In [ ]:
num = []
cat = []
data.drop('index',axis = 1,inplace = True)
for i in data.columns:
    if (data[i].dtype in numeric_dtypes)&(i != 'SalePrice'):
            num.append(i)  
    elif i != 'SalePrice': #elif train_data[i].dtype=='object'
            cat.append(i) 
change = ['YearBuilt_Before1950','YearBuilt_After1980','YearRemodel_before1990']
for c in change:
    num.remove(c)
    cat.insert(-1,c)
num.remove('lable')
data['MSZoning'][data['MSZoning']=='C (all)'] = ['C']*len(data['MSZoning'][data['MSZoning']=='C (all)'])

In [ ]:
data['RoofMatl'][data['RoofMatl']=='Tar&Grv'] = ['TarGrv']*len(data['RoofMatl'][data['RoofMatl']=='Tar&Grv'])

In [ ]:
#data.to_csv('Before_R_defore_dummy.csv')

In [ ]:
# Dummy variable (one-hot) 
dummy = cat.copy()
dummy.remove('YearBuilt_Before1950')
dummy.remove('YearBuilt_After1980')
dummy.remove('YearRemodel_before1990')
data = data.join(pd.get_dummies(data[dummy], drop_first = True), how = 'outer')
data.drop(dummy,axis = 1, inplace = True)

In [ ]:
data.to_csv('Before_R.csv')

In [ ]:
# 分割
train_data = data[data['lable']==1]
test_data = data[data['lable']==0]
train_data.drop('lable', axis = 1, inplace = True)
x_train = train_data.drop('SalePrice',axis = 1)
y_train = train_data['SalePrice']

In [ ]:
num = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'stFlrSF',
         'ndFlrSF', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageArea', 'WoodDeckSF', 'PoolArea', 'MiscVal', 'UnfinishedBsm_ratio', 'LowQuality_ratio', 'Bath_total', 'Porch', 'Com1', 'Com2', 'Com3', 'Com4', 'Com5', 'Com6', 'Com7', 'Com8']

In [ ]:
data.head()

**检查相关性**

In [ ]:
plt.figure(figsize = (15,15))
N = num.copy()
N.insert(len(N),'SalePrice')
sns.heatmap(train_data.loc[:, N].corr(method = 'spearman'),annot=True)

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(train_data.loc[:, N].corr(),annot=True)

In [ ]:
plt.scatter(x_train['LotArea'], np.log(y_train), c = "blue", marker = "s", label = "Training data")

怪怪的

**QQ-Plot**

ln(Y)变量的正态性检验

In [ ]:
sm.qqplot(np.log(y_train), line = 's')
pylab.show()

In [ ]:
# Standardize numerical features
stdSc = StandardScaler()
x_train.loc[:, num] = stdSc.fit_transform(x_train.loc[:, num])

离群值暂时没搞

**粗略回归**

In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(x_train, np.log(y_train))

# Look at predictions on training and validation set
y_train_pred = lr.predict(x_train)

# Plot predictions
plt.scatter(y_train_pred, np.log(y_train), c = "blue", marker = "s", label = "Training data")
plt.title("Linear regression")
plt.xlabel("Predicted values")
plt.ylabel("Real values")
plt.legend(loc = "upper left")
plt.plot([10.5, 13.5], [10.5, 13.5], c = "red")
plt.show()

In [ ]:
model = sm.OLS(np.log(y_train),x_train)
results = model.fit()
print(results.summary())

In [ ]:
results.pvalues.index.values[results.pvalues.argmax()]

In [ ]:
import statsmodels.formula.api as smf
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by AIC
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 10000000, 10000000
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            #formula = "{} ~ {} + 1".format(response,
            #                               ' + '.join(selected + [candidate]))
            score = sm.OLS(data[response], data[selected + [candidate]]).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort(reverse = True)
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        print(best_new_score)
        print(len(remaining))
    #formula = "{} ~ {} + 1".format(response,
     #                              ' + '.join(selected))
    model = sm.OLS(data[response], data[selected + [candidate]]).fit()
    return model

In [ ]:
try_data = pd.concat([x_train,y_train], axis = 1)
model_try = forward_selected(try_data, 'SalePrice')

In [ ]:
def stepwise_selection(X, y, 
                       initial_list = [], 
                       threshold_in = 0.03, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index.values[new_pval.argmin()]
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


In [ ]:
a = stepwise_selection(x_train, y_train,initial_list=[])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
aic = pd.read_csv('/kaggle/input/aicmodel/stp_coef.csv')
model2 = pd.read_csv('/kaggle/input/model2-coef/model2_coef.csv')

In [ ]:
sd = {'MSSubClass': 41.2525762994777,
      'LotFrontage': 22.1703048984145, 'LotArea': 10336.6211264189, 'OverallQual':1.32447167128337,
      'OverallCond': 1.07812365756163, 'MasVnrArea':185.604815717681,
      'stFlrSF':386.644985557987,'ndFlrSF':440.324982234229, 'BedroomAbvGr': 0.776677111539577,
      'KitchenAbvGr': 0.174697484763521,'TotRmsAbvGrd':1.58507082120861,'Fireplaces':0.645392646346361,
      'GarageArea':186.761862747612,'WoodDeckSF':127.537064801993,'PoolArea':41.9613371705121,
      'MiscVal':508.056254538568, 'UnfinishedBsm_ratio':0.359890753307336,
      'LowQuality_ratio':0.02354720865502,'Bath_total':0.410507389307702,'Porch':105.488091895556,
      'Com1': 3799.39793184623, 'Com2': 1606.65158970446, 'Com3': 392.15867923776,'Com4':1239206.38313568,
      'Com5': 3.87402016859075, 'Com6': 4.37511010349748, 'Com7': 34720.9475251887, 'Com8': 9.23769982728718}

In [ ]:
sd = pd.DataFrame([sd]).T
sd.columns = ['sd']

In [ ]:
aic.columns=['aic','coef']
aic.set_index('aic',inplace=True)

In [ ]:
model2.columns=['model2','coef']
model2.set_index('model2',inplace=True)

In [ ]:
aic = pd.merge(aic,sd, left_index=True,right_index=True, how = 'left')
model2 = pd.merge(model2,sd, left_index=True,right_index=True, how = 'left')

In [ ]:
model2 = model2.fillna(1)
aic = aic.fillna(1)
aic['Coef'] = aic['coef'] * aic['sd']
model2['Coef'] = model2['coef'] * model2['sd']

In [ ]:
sort_model2 = abs(model2).sort_values(by = 'Coef')

In [ ]:
sort_aic = abs(aic).sort_values(by = 'Coef')

In [ ]:
for i in range(sort_aic.shape[0]):
        print(sort_aic.index[i], aic.loc[sort_aic.index[i],'Coef'])

In [ ]:
print('Foundation_Wood &', 'Foundation_Stone')
print(np.exp(0.0714736608810204 +0.211679508247599))

In [ ]:
print('RoofMatl_Membran &', 'RoofMatl_Metal, ', 'RoofMatl_WdShngl')
print(np.exp(2.53605872693327 - 2.48098919258797))
print(np.exp(2.53605872693327 - 2.35482340515519))

In [ ]:
print('HeatingQC_Po &','HeatingQC_Gd' )
print(np.exp( -0.0306984003361131 + 0.117812833850804))

In [ ]:
for i in range(sort_model2.shape[0]):
        print(sort_model2.index[i], model2.loc[sort_model2.index[i],'Coef'])

In [ ]:
aic_cat = ['RoofMatl_Membran', 'MSZoning_FV', 'Functional_Sev', 'Foundation_Wood', 
'SaleCondition_AdjLand','Utilities_NoSeWa','RoofStyle_Shed', 'LandSlope_Sev','SaleType_Oth']
aic_num = ['Com6', 'stFlrSF', 'Com2', 'ndFlrSF', 'TotRmsAbvGrd']
aic_cat.extend(aic_num)
aic_draw = aic.loc[aic_cat, 'coef'].sort_values()

In [ ]:
model2_cat = ['RoofMatl_Membran', 'MSZoning_FV', 'Functional_Sev', 'SaleCondition_AdjLand',
             'Foundation_Wood', 'LandSlope_Sev', 'HeatingQC_Po', 'KitchenQual_Fa']
model2_num = ['stFlrSF', 'Com6', 'ndFlrSF', 'OverallQual', 'Bath_total']
model2_cat.extend(model2_num)
model2_draw = model2.loc[model2_cat, 'coef'].sort_values()

In [ ]:
model2.loc[model2_num,'sd']

In [ ]:
np.exp(model2.loc[model2_num,'coef'])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
aic_draw.plot(kind = "barh", xlim = [-0.5, 2.8])
plt.title("Coefficients in the AIC modle")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
model2_draw.plot(kind = "barh", xlim = [-0.5, 3.5])
plt.title("Coefficients in the WLS modle")